# Web Scraping con Python aplicado al sitio web del INEGI

Empezamos por analizar el sitio web con la ayuda de las herramientas de desarrollados del navegador web.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import requests_cache

Esta herramienta es particularmente útil cuando nos encontramos en fase de pruebas del código, ya que ayuda a lidiar con el firewall del servidor.

In [2]:
requests_cache.install_cache('inegi_cache')

La Red Nacional de Metadatos cuenta con un catálogo en formato csv, lo utilizaremos para filtrar la búsqueda

In [3]:
catalogue = pd.read_csv('https://www.inegi.org.mx/rnm/index.php/catalog/export/csv?ps=5000&collection[]=')

exploramos el contenido del catálogo

In [4]:
catalogue.head()

,id,idno,title,nation,authoring_entity,year_start,year_end,created,changed
0,641,MEX-INEGI.EEC6.03-BCMM-2021,Balanza Comercial de Mercancías de México 2021...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-09-21,Sep-08-21
1,634,MEX-INEGI-EEC3.01-EMOE-2021,"Encuesta Mensual de Opinión Empresarial 2021, ...",Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Jan-29-21,Aug-31-21
2,650,MEX-INEGI.EEC3.02-EMEC-2021,Encuesta Mensual sobre Empresas Comerciales 20...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-24-21,Aug-24-21
3,648,MEX-INEGI.EEC3.01-ENEC-2021,Encuesta Nacional de Empresas Constructoras 20...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-22-21,Aug-26-21
4,689,MEX-INEGI.ESD3.03-ENOEN-2021-II,Encuesta Nacional de Ocupación y Empleo Nueva ...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Aug-18-21,Aug-26-21


por ahora, nos interesa los archivos más recientes, por lo que nos enfocaremos en la columna "change", la convertimos al tipo datetime

In [5]:
catalogue['changed'] = pd.to_datetime(catalogue['changed'])

convertimos la columna "id" como índice

In [6]:
catalogue.set_index('id')

,idno,title,nation,authoring_entity,year_start,year_end,created,changed
id,,,,,,,,
641,MEX-INEGI.EEC6.03-BCMM-2021,Balanza Comercial de Mercancías de México 2021...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-09-21,2021-09-08
634,MEX-INEGI-EEC3.01-EMOE-2021,"Encuesta Mensual de Opinión Empresarial 2021, ...",Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Jan-29-21,2021-08-31
650,MEX-INEGI.EEC3.02-EMEC-2021,Encuesta Mensual sobre Empresas Comerciales 20...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-24-21,2021-08-24
648,MEX-INEGI.EEC3.01-ENEC-2021,Encuesta Nacional de Empresas Constructoras 20...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-22-21,2021-08-26
689,MEX-INEGI.ESD3.03-ENOEN-2021-II,Encuesta Nacional de Ocupación y Empleo Nueva ...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Aug-18-21,2021-08-26
...,...,...,...,...,...,...,...,...
278,MEX-INEGI.ESD3.06-EVMAT-1993-2001,Estadísticas Vitales. Matrimonios 1993-2001,Mexico,Instituto Nacional de Estadística y Geografía ...,1993,2001,Aug-07-17,2017-08-07
295,MEX-INEGI.EST.40.202.05-DERL-1991-2016,Estadísticas sobre Relaciones Laborales de Jur...,Mexico,"Instituto Nacional de Estadística y Geografía,...",1991,2016,Nov-01-17,2020-10-16
382,MEX-INEGI.302.03-VMRC-1991-2016,"Vehículos de Motor Registrados en Circulación,...",Mexico,"Instituto Nacional de Estadística y Geografía,...",1991,2017,Sep-12-18,2018-09-12


ordenamos la tabla por la columna "changed" empezando por lo más recientes

In [8]:
catalogue.sort_values(by = 'changed', ascending = False, inplace=True)
catalogue.head(10)

,id,idno,title,nation,authoring_entity,year_start,year_end,created,changed
48,694,MEX-INEGI.ESD5.03-EVM-2020,Estadísticas Vitales. Estadísticas de Defuncio...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2020,2020,Aug-30-21,2021-09-15
8,640,MEX-INEGI.EEC3.04-EVI-EVF-2021,"Encuestas de Viajeros Internacionales 2021, En...",Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-09-21,2021-09-13
11,644,MEX-INEGI.EEC6.03-ETUP-2021,Estadística de Transporte Urbano de Pasajeros ...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-12-21,2021-09-13
80,470,MEX-INEGI-EEC3.01-EMOE-2019,"Encuesta Mensual de Opinión Empresarial 2019, ...",Mexico,"Instituto Nacional de Estadística y Geografía,...",2019,2020,Mar-29-19,2021-09-13
17,637,MEX-INEGI.EEC6.03-VMRC-2021,Vehículos de Motor Registrados en Circulación ...,Mexico,Instituto Nacional de Estadística y Geografía ...,2021,2021,Feb-24-21,2021-09-10
20,697,MEX-INEGI.40.1200.03-CNIJF-2021,Censo Nacional de Impartición de Justicia Fede...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2020,2021,Sep-06-21,2021-09-09
7,639,MEX-INEGI.EEC3.04-EVI-ETI-2021,"Encuestas de Viajeros Internacionales 2021, En...",Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-09-21,2021-09-09
0,641,MEX-INEGI.EEC6.03-BCMM-2021,Balanza Comercial de Mercancías de México 2021...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-09-21,2021-09-08
35,695,MEX-INEGI.ESD3.04-ENVI-2020,Encuesta Nacional de Vivienda (ENVI) 2020,Mexico,"Instituto Nacional de Estadística y Geografía,...",2020,2021,Sep-01-21,2021-09-07
37,674,MEX-INEGI.EEC3.04-ENDUTIH-2020,Encuesta Nacional sobre Disponibilidad y Uso d...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2020,2021,Jun-21-21,2021-09-07


filtramos por fecha para explorar en los archivos publicados en la última semana

In [9]:
sep2021 = catalogue.loc[catalogue['changed'].between('2021-09-09', '2021-09-09')]
sep2021.head()

,id,idno,title,nation,authoring_entity,year_start,year_end,created,changed
20,697,MEX-INEGI.40.1200.03-CNIJF-2021,Censo Nacional de Impartición de Justicia Fede...,Mexico,"Instituto Nacional de Estadística y Geografía,...",2020,2021,Sep-06-21,2021-09-09
7,639,MEX-INEGI.EEC3.04-EVI-ETI-2021,"Encuestas de Viajeros Internacionales 2021, En...",Mexico,"Instituto Nacional de Estadística y Geografía,...",2021,2021,Mar-09-21,2021-09-09


creamos un ciclo para descargar los archivos de cada uno de los "id" filtrados en el paso anterior

In [15]:
url = 'https://www.inegi.org.mx/rnm/index.php/catalog/'
for itera in sep2021.id:
    response = requests.get(url + str(itera) + '/related-materials') #realiza la petición al sitio
    print('Response code: ', response.status_code) #imprime la respuesta del sitio
    soup = BeautifulSoup(response.text, 'html.parser') #extrae la información del sitio
    links = soup.find_all('a', class_ = 'download btn btn-outline-primary btn-sm btn-block') #busca las etiquetas "a" y "class"
    time.sleep(3) #pausa el código por unos segundos
    #crea un nuevo ciclo para descargar los archivos
    i = 0
    for link in links:
        i += 1
        print('Downloading file: ', i)
        response = requests.get(link.get('href'))
        liga = link.get('href')
        filename = liga.split('/')[-1]
        file = open(str(itera) + '-'+ str(i) +'_'+ str(filename), 'wb') #lee el archivo JSON
        file.write(response.content) #salva el archivo
        file.close()
        print('File ', itera, i, filename, ' downloaded')
        time.sleep(3)
print("All files downloaded")

Response code:  200
File  697 1 cnijf_2021_m1s1.pdf  downloaded
File  697 2 cnijf_2021_m1s2.pdf  downloaded
File  697 3 cnijf_2021_m1s3.pdf  downloaded
File  697 4 cnijf_2021_m2.pdf  downloaded
File  697 5 cnijf_2021_m3.pdf  downloaded
File  697 6 cnijf_2021_m4.pdf  downloaded
File  697 7 #Tabulados  downloaded
File  697 8 cnijf_2021_resultados.pdf  downloaded
File  697 9 #Datos_abiertos  downloaded
File  697 10 889463901570.pdf  downloaded
Response code:  200
File  639 1 T_1.pdf  downloaded
File  639 2 T_1_T.pdf  downloaded
File  639 3 T_2.pdf  downloaded
File  639 4 T_2_T.pdf  downloaded
File  639 5 702825104368.pdf  downloaded
File  639 6 19366  downloaded
File  639 7 19367  downloaded
File  639 8 19368  downloaded
All files downloaded


creamos un ciclo para explorar las ligas y buscar archivos en formatos JSON y XLM, aprovechamos que el url está compuesto de una url base + el id del catálogo

In [14]:
url = 'https://www.inegi.org.mx/rnm/index.php/catalog/'
for ii in sep2021.id:
    response = requests.get(url + str(ii) + '/related-materials') #realiza la petición al sitio
    print(response.status_code) #imprime la respuesta del sitio
    soup = BeautifulSoup(response.text, 'html.parser') #extrae la información del sitio
    links = soup.find_all('a') #busca las etiquetas con hipervínculo
    time.sleep(3) #pausa el código por unos segundos
    #crea un nuevo ciclo para buscar archivos JSON y XML
    i = 0
    for link in links:
        if ('/json' in link.get('href', [])): #filtra la busqueda a archivos JSON
            i += 1
            print('Downloading file: ', ii, i)
            response = requests.get(link.get('href'))
            json = open('json' + str(ii) + '_' + str(i) + '.json', 'wb') #lee el archivo JSON
            json.write(response.content) #salva el archivo
            json.close()
            print('File ', ii, i, ' downloaded')
            time.sleep(3)
        elif ('/ddi' in link.get('href', [])):
            i += 1
            print('Downloading file: ', ii, i)
            response = requests.get(link.get('href'))
            xml = open('xml' + str(ii) + str(i) + '.xml', 'wb')
            xml.write(response.content)
            xml.close()
            print('File ', i, ' downloaded')
            time.sleep(3)
        
print("All files downloaded")

200
Response id 697
File  1  downloaded
File  697 2  downloaded
200
Response id 639
File  1  downloaded
File  639 2  downloaded
All files downloaded


dejo pendiente crear y descargar a una carpeta, extraer el "content-disposition" para los nombre de los archivos, tomar la fecha del SO para bajar los archivos publicados el día en que se ejecuta el programa.

### Referencias:
Instituto Nacional de Estadística Geografía e Informática(s.f). Red Nacional de Metadatos. https://www.inegi.org.mx/rnm/index.php/catalog

Wulf, P. (25 de marzo de 2021). Web Scraping with Beautiful Soup. https://www.scrapingbee.com/blog/python-web-scraping-beautiful-soup/

Ujhelyi, T. (29 de marzo de 2021). Beautiful Soup Tutorial 1. https://data36.com/beautiful-soup-tutorial-web-scraping/

Downloading PDFs with Python using Requests and Beautiful Soup. (13 de abril de 2021). https://www.geeksforgeeks.org/downloading-pdfs-with-python-using-requests-and-beautifulsoup/?ref=rp